## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-31-03-52-59 +0000,bbc,Letting agent apologises for 'oversight' on Re...,https://www.bbc.com/news/articles/cx2p55ejy88o...
1,2025-10-31-03-49-00 +0000,wsj,Disney Channels Go Dark on YouTube TV,https://www.wsj.com/business/media/disney-chan...
2,2025-10-31-03-35-37 +0000,nypost,Noem warns trucks are becoming ‘weapons’ as Ne...,https://nypost.com/2025/10/30/us-news/noem-war...
3,2025-10-31-03-29-29 +0000,nyt,Ohio Republicans Gain Ground in Push for More ...,https://www.nytimes.com/2025/10/30/us/politics...
4,2025-10-31-03-24-00 +0000,wsj,China Factory Activity Gauge Signals Deepening...,https://www.wsj.com/economy/china-factory-acti...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
158,trump,64
31,china,21
279,new,19
67,trade,17
275,xi,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
35,2025-10-31-00-58-02 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...,151
242,2025-10-30-16-06-07 +0000,bbc,Trump hails 'amazing' meeting with China's Xi ...,https://www.bbc.com/news/articles/crl25xl1gjpo...,147
363,2025-10-30-08-56-55 +0000,cbc,Trump sets new China tariff rate after 'amazin...,https://www.cbc.ca/news/world/us-china-trump-j...,137
281,2025-10-30-13-36-51 +0000,wapo,Trump cuts tariffs on China after ‘truly great...,https://www.washingtonpost.com/politics/2025/1...,130
62,2025-10-30-23-41-00 +0000,wsj,A day after President Trump vowed to resume te...,https://www.wsj.com/politics/national-security...,128


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
35,151,2025-10-31-00-58-02 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...
61,59,2025-10-30-23-42-12 +0000,nypost,Amazon shares soar as AI demand boosts cloud r...,https://nypost.com/2025/10/30/business/amazon-...
31,55,2025-10-31-01-03-50 +0000,nyt,"In China Truce on Tariffs and Rare Earths, Nat...",https://www.nytimes.com/2025/10/30/us/politics...
28,52,2025-10-31-01-17-54 +0000,nyt,Jamaica Prepared a Financial Fortress for Disa...,https://www.nytimes.com/2025/10/30/world/ameri...
230,52,2025-10-30-16-29-00 +0000,wsj,Crocs Focuses on Long-Term Growth as Third-Qua...,https://www.wsj.com/business/earnings/crocs-pr...
62,51,2025-10-30-23-41-00 +0000,wsj,A day after President Trump vowed to resume te...,https://www.wsj.com/politics/national-security...
176,45,2025-10-30-19-35-06 +0000,nypost,Disgraced Prince Andrew to be stripped of ‘pri...,https://nypost.com/2025/10/30/world-news/princ...
192,44,2025-10-30-18-40-15 +0000,nypost,Kamala Harris claims Biden ‘didn’t want that d...,https://nypost.com/2025/10/30/us-news/kamala-h...
182,41,2025-10-30-19-14-03 +0000,nypost,Hamas hostages released to Israel ID’d as 84-y...,https://nypost.com/2025/10/30/world-news/hamas...
121,40,2025-10-30-21-34-58 +0000,nyt,Do You Rely on Food Stamps? Tell Us About How ...,https://www.nytimes.com/2025/10/30/us/snap-foo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
